In [1]:
import os
import sys
import zipfile
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

%matplotlib inline

sys.path.append('../')
from src.data_preprocess import DataPreprocessor

pd.set_option('display.max_colwidth', 2000)
import warnings
warnings.filterwarnings('ignore')

%load_ext autoreload
%autoreload 2

In [2]:
train_df = pd.read_csv('../input/train.csv')
train_df.head(2)

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
0,122881,hdb flat for sale in 866 yishun street 81,sembawang / yishun (d27),866 yishun street 81,hdb 4 rooms,NaN,1988.0,3.0,2.0,1115,...,unspecified,NaN,116.0,https://www.99.co/singapore/hdb/866-yishun-street-81-adxawp85buupmsq7gwdjverc,1.414399,103.837196,0,yishun south,yishun,514500.0
1,259374,hdb flat for sale in 506b serangoon north avenue 4,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,99-year leasehold,1992.0,4.0,2.0,1575,...,unspecified,"1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbserangoon-estate-demrpm6ryc3l9buf846erprb,1.372597,103.875625,0,serangoon north,serangoon,995400.0


In [3]:
# print(train_df.shape)
# display(train_df.isnull().sum())
train_df_clean = DataPreprocessor.data_preprocessing_v1(train_df, test=False, uncertain=False, drop_na=False, remove_original_attributes=True)
# print(train_df_clean.shape)
# display(train_df_clean.isnull().sum())

Processed function: Function 'preprocess_available_unit_types' executed in 5.1245s & Processing function: preprocess_planning_area: 100%|██████████| 14/14 [00:42<00:00,  3.00s/it]


In [4]:
display(train_df_clean.isnull().sum())

built_year                   494
num_beds                       0
num_baths                      1
size_sqft                      0
lat                            0
lng                            0
price                          0
property_type_cat_0            0
tenure_cat_0                   0
tenure_cat_1                   0
tenure_cat_2                   0
tenure_cat_3                   0
tenure_cat_4                   0
total_level_cat_0              0
floor_level_cat_0              0
floor_level_cat_1              0
floor_level_cat_2              0
floor_level_cat_3              0
floor_level_cat_4              0
floor_level_cat_5              0
floor_level_cat_6              0
floor_level_cat_7              0
furnishing_cat_0               0
furnishing_cat_1               0
furnishing_cat_2               0
furnishing_cat_3               0
number_of_types_available      0
has_studio                     0
min_br_available               0
max_br_available               0
planning_a

In [5]:
# train_df_model = train_df_clean.drop(['lat', 'lng'], axis=1,inplace=False).reset_index(drop=True)
train_df_model = train_df_clean
train_df_model.head()

,built_year,num_beds,num_baths,size_sqft,lat,lng,price,property_type_cat_0,tenure_cat_0,tenure_cat_1,...,floor_level_cat_7,furnishing_cat_0,furnishing_cat_1,furnishing_cat_2,furnishing_cat_3,number_of_types_available,has_studio,min_br_available,max_br_available,planning_area_cat_0
0,2022.0,1.0,1.0,495,1.31629,103.840576,1424800.0,5.0,0.0,0.0,...,1.0,0.0,1.0,0.0,0.0,4,1,1,3,0.0
1,2022.0,1.0,1.0,495,1.31629,103.840576,1433200.0,5.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,4,1,1,3,0.0
2,2022.0,1.0,1.0,495,1.31629,103.840576,1424800.0,6.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,4,1,1,3,0.0
3,2022.0,1.0,1.0,495,1.31629,103.840576,1424800.0,5.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,4,1,1,3,0.0
4,2022.0,1.0,1.0,495,1.31629,103.840576,1390700.0,5.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,4,1,1,3,0.0


In [6]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score
from hyperopt import fmin, hp, tpe, STATUS_OK, STATUS_FAIL, Trials
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold

train_df_model_nonNa = train_df_model.dropna()

X_nonNa = train_df_model_nonNa.drop('price', axis=1,inplace=False).astype(float)
y_nonNa = train_df_model_nonNa['price'].astype(float)
X = train_df_model.drop('price', axis=1,inplace=False).astype(float)
y = train_df_model['price'].astype(float)

In [6]:
# from sklearn.tree import DecisionTreeRegressor
# from explainerdashboard import RegressionExplainer

# X_train, X_test, y_train, y_test = train_test_split(X_nonNa, y_nonNa)
# decision_reg = DecisionTreeRegressor(max_depth=10)
# decision_reg.fit(X_train, y_train)

# explainer = RegressionExplainer(decision_reg, X_test, y_test)
# from explainerdashboard import ExplainerDashboard
# ExplainerDashboard(explainer).run()

In [7]:
# from sklearn.tree import DecisionTreeRegressor
# from explainerdashboard import RegressionExplainer

# X_train, X_test, y_train, y_test = train_test_split(X, y)

# # xgb_regressor = xgb.XGBRegressor(
# #     n_estimators =115, 
# #     max_depth = 15, 
# #     gamma = 8.973856858008121,
# #     reg_alpha = 115,
# #     min_child_weight=41,
# #     colsample_bytree=0.4429943486665428)

# xgb_regressor = xgb.XGBRegressor(
#     n_estimators =210, 
#     max_depth = 8, 
#     gamma = 4.404607622809311,
#     reg_alpha = 73,
#     min_child_weight=41,
#     colsample_bytree=0.5856071130141778)

# xgb_regressor.fit(X_train, y_train)

# explainer = RegressionExplainer(xgb_regressor, X_test, y_test)
# from explainerdashboard import ExplainerDashboard
# ExplainerDashboard(explainer).run()

# test prediction / for submission

In [7]:
test_df = pd.read_csv('../input/test.csv')
test_df.head(1)

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area
0,777912,1 bed condo for sale in the gazania,17 how sun drive,the gazania,condo,freehold,2022.0,1.0,1.0,463,NaN,unfurnished,"studio, 1, 2, 3, 4, 5 br",250.0,https://www.99.co/singapore/condos-apartments/the-gazania,1.344334,103.87869,0,upper paya lebar,serangoon


In [8]:
# print(test_df.shape)
# display(test_df.isnull().sum())
test_df_clean = DataPreprocessor.data_preprocessing_v1(test_df, test=True, uncertain=False, drop_na=False, remove_original_attributes=True)
# print(test_df_clean.shape)
# display(test_df_clean.isnull().sum())

(7000, 20)


listing_id                 0
title                      0
address                    2
property_name              0
property_type              0
tenure                   637
built_year               358
num_beds                  35
num_baths                152
size_sqft                  0
floor_level             5844
furnishing                 0
available_unit_types     520
total_num_units         1900
property_details_url       0
lat                        0
lng                        0
elevation                  0
subzone                   33
planning_area             33
dtype: int64

Processed function: Function 'preprocess_available_unit_types' executed in 1.7780s & Processing function: preprocess_planning_area: 100%|██████████| 14/14 [00:22<00:00,  1.58s/it]

(7000, 30)


built_year                   291
num_beds                       0
num_baths                      0
size_sqft                      0
lat                            0
lng                            0
property_type_cat_0            0
tenure_cat_0                   0
tenure_cat_1                   0
tenure_cat_2                   0
tenure_cat_3                   0
tenure_cat_4                   0
total_level_cat_0              0
floor_level_cat_0              0
floor_level_cat_1              0
floor_level_cat_2              0
floor_level_cat_3              0
floor_level_cat_4              0
floor_level_cat_5              0
floor_level_cat_6              0
floor_level_cat_7              0
furnishing_cat_0               0
furnishing_cat_1               0
furnishing_cat_2               0
furnishing_cat_3               0
number_of_types_available      0
has_studio                     0
min_br_available               0
max_br_available               0
planning_area_cat_0            0
dtype: int

In [10]:
test_df_clean.head()

,built_year,num_beds,num_baths,size_sqft,lat,lng,property_type_cat_0,tenure_cat_0,tenure_cat_1,tenure_cat_2,...,floor_level_cat_7,furnishing_cat_0,furnishing_cat_1,furnishing_cat_2,furnishing_cat_3,number_of_types_available,has_studio,min_br_available,max_br_available,planning_area_cat_0
0,2022.0,1.0,1.0,463,1.344334,103.878690,5.0,0.0,0.0,1.0,...,1.0,1.0,0.0,0.0,0.0,6,1,1,5,3.0
1,2017.0,3.0,3.0,1033,1.380281,103.943878,5.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,6,1,1,5,14.0
2,2007.0,1.0,3.0,570,1.294668,103.850074,5.0,0.0,1.0,0.0,...,1.0,0.0,0.0,1.0,0.0,4,1,1,3,19.0
3,2017.0,3.0,2.0,1216,1.373120,103.746094,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,5,0,1,5,34.0
4,1973.0,3.0,2.0,936,1.341468,103.849047,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,4,0,1,4,2.0


In [34]:
# xgb_regressor = xgb.XGBRegressor(
#     n_estimators =115, 
#     max_depth = 15, 
#     gamma = 8.973856858008121,
#     reg_alpha = 115,
#     min_child_weight=41,
#     colsample_bytree=0.4429943486665428)
# xgb_regressor = xgb.XGBRegressor(
#     n_estimators =210, 
#     max_depth = 8, 
#     gamma = 4.404607622809311,
#     reg_alpha = 73,
#     min_child_weight=41,
#     colsample_bytree=0.5856071130141778)

# best!!
params = {'colsample_bytree': 0.9988638693168667,
 'gamma': 4.626267990192529,
 'max_depth': 10,
 'min_child_weight': 29,
 'n_estimators': 440,
 'reg_alpha': 148}

xgb_regressor = xgb.XGBRegressor(**params)

# # define model evaluation method
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# # evaluate model
# scores = cross_val_score(xgb_regressor, X, y, scoring='r2', cv=cv, n_jobs=-1)
# scores

In [35]:
xgb_regressor.fit(X, y)

test_df_model = test_df_clean#.drop(['lat', 'lng'], axis=1,inplace=False)
X_test = test_df_model.astype(float)
output = pd.DataFrame(xgb_regressor.predict(X_test)).reset_index().rename(columns={"index": "Id", 0: "Predicted"})

In [36]:
output.head()

,Id,Predicted
0,0,1.174768e+06
1,1,1.507884e+06
2,2,2.221022e+06
3,3,7.661581e+05
4,4,5.744552e+05


In [37]:
output.to_csv('submission.csv', index=False)

# Hyperopt

In [16]:
import warnings
warnings.filterwarnings('ignore')
# XGB parameters
xgb_reg_params={
    'n_estimators': hp.quniform('n_estimators', 100, 500, 10),
    'max_depth': hp.quniform("max_depth", 5, 20, 1),
    'gamma': hp.uniform('gamma', 1,9),
    'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
    'colsample_bytree' : hp.uniform('colsample_bytree', 0.2,1),
    'min_child_weight' : hp.quniform('min_child_weight', 10, 50, 1),
}
xgb_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 50,
    'verbose': False
}
xgb_para = dict()
xgb_para['reg_params'] = xgb_reg_params
xgb_para['fit_params'] = xgb_fit_params
xgb_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))
xgb_para['score_func' ] = lambda y, pred: r2_score(y, pred)

# Random Forest
random_forest_reg_params={
    'n_estimators':hp.uniform('n_estimators',100,500),
    'max_depth':hp.uniform('max_depth',5,50),
    'min_samples_leaf':hp.uniform('min_samples_leaf',1,5),
    'min_samples_split':hp.uniform('min_samples_split',2,6)}
random_forest_fit_params = {
    'eval_metric': 'rmse',
    'early_stopping_rounds': 10,
    'verbose': True
}
random_forest_para = dict()
random_forest_para['reg_params'] = random_forest_reg_params
random_forest_para['fit_params'] = random_forest_fit_params
random_forest_para['loss_func' ] = lambda y, pred: np.sqrt(mean_squared_error(y, pred))
random_forest_para['score_func' ] = lambda y, pred: r2_score(y, pred)



class HPOpt(object):

    def __init__(self, x_train, x_test, y_train, y_test):
        self.x_train = x_train
        self.x_test  = x_test
        self.y_train = y_train
        self.y_test  = y_test

    def process(self, fn_name, space, trials, algo, max_evals):
        fn = getattr(self, fn_name)
        try:
            result = fmin(fn=fn, space=space, algo=algo, max_evals=max_evals, trials=trials)
        except Exception as e:
            return {'status': STATUS_FAIL,
                    'exception': str(e)}
        return result, trials

    def xgb_reg(self, para):
        space = para['reg_params']
        reg = xgb.XGBRegressor(
            n_estimators =int(space['n_estimators']), 
            max_depth = int(space['max_depth']),
            gamma = float(space['gamma']),
            reg_alpha = int(space['reg_alpha']),
            colsample_bytree=float(space['colsample_bytree']),
            min_child_weight=int(space['min_child_weight']),
            n_jobs = 4
        )
        return self.train_reg(reg, para)

    def random_forest_reg(self, para):
        space = para['reg_params']
        reg = RandomForestRegressor(
            n_estimators=int(space['n_estimators']),
            max_depth=int(space['max_depth']),
            min_samples_leaf=int(space['min_samples_leaf']),
            min_samples_split=int(space['min_samples_split']),
            n_jobs=-1
        )
        return self.train_reg(reg, para)

#     def ctb_reg(self, para):
#         reg = ctb.CatBoostRegressor(**para['reg_params'])
#         return self.train_reg(reg, para)

    def train_reg(self, reg, para):
        # reg.fit(self.x_train, self.y_train)
        reg.fit(self.x_train, self.y_train,
                eval_set=[(self.x_train, self.y_train), (self.x_test, self.y_test)],
                **para['fit_params'])
        pred = reg.predict(self.x_test)
        loss = para['loss_func'](self.y_test, pred)
        score = para['score_func'](self.y_test, pred)
        return {'loss': loss, 'score': score,'status': STATUS_OK}

# X_train, X_test, y_train, y_test = train_test_split(X, y)
# obj = HPOpt(X_train, X_test, y_train, y_test)
# xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=500)
# best loss: 2271478.595400419
# X_train, X_test, y_train, y_test = train_test_split(X_nonNa, y_nonNa)
# obj = HPOpt(X_train, X_test, y_train, y_test)
# random_forest_opt = obj.process(fn_name='random_forest_reg', space=random_forest_para, trials=Trials(), algo=tpe.suggest, max_evals=100)

In [18]:
# space = xgb_opt[0]
# space

In [40]:
# {'colsample_bytree': 0.5856071130141778,
#  'gamma': 4.404607622809311,
#  'max_depth': 8.0,
#  'min_child_weight': 14.0,
#  'n_estimators': 210.0,
#  'reg_alpha': 73.0}

In [19]:
# xgb_regressor = xgb.XGBRegressor(
#     n_estimators =int(space['n_estimators']), 
#     max_depth = int(space['max_depth']), 
#     gamma = float(space['gamma']),
#     reg_alpha = int(space['reg_alpha']),
#     min_child_weight=int(space['min_child_weight']),
#     colsample_bytree=float(space['colsample_bytree']))

# # define model evaluation method
# cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# # evaluate model
# scores = cross_val_score(xgb_regressor, X, y, scoring='r2', cv=cv, n_jobs=-1)
# scores

In [20]:
# non-optimzed: r2= 0.7817454737515114
# first opt: r2= 0.8347165176902147
# np.mean(scores)

In [30]:
from IPython.display import clear_output

best_score = 0.8389878760580836
score = 0
score_temp = 0
counter = 0
best_settings = {}
while best_score > score:
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    obj = HPOpt(X_train, X_test, y_train, y_test)
    xgb_opt = obj.process(fn_name='xgb_reg', space=xgb_para, trials=Trials(), algo=tpe.suggest, max_evals=300)
    space = xgb_opt[0]
    xgb_regressor = xgb.XGBRegressor(
        n_estimators =int(space['n_estimators']), 
        max_depth = int(space['max_depth']), 
        gamma = float(space['gamma']),
        reg_alpha = int(space['reg_alpha']),
        min_child_weight=int(space['min_child_weight']),
        colsample_bytree=float(space['colsample_bytree']))

    # define model evaluation method
    cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
    # evaluate model
    scores = cross_val_score(xgb_regressor, X, y, scoring='r2', cv=cv, n_jobs=-1)
    clear_output(wait=True)
    if np.mean(scores) > score:
        best_settings = xgb_opt[0]
        score_temp = np.mean(scores)
        print(f"score update = {score_temp}")
        if np.round(score_temp, 3) > np.round(best_score, 3):
            best_score = score_temp
            break
    if counter % 5 == 0:
        print(f"current score = {score_temp} vs best score = {best_score}")
    if counter == 100: break
    counter += 1

score update = 0.8408482185349724


In [31]:
space = xgb_opt[0]
space

{'colsample_bytree': 0.647778961827119,
 'gamma': 3.6721626294320266,
 'max_depth': 10.0,
 'min_child_weight': 44.0,
 'n_estimators': 160.0,
 'reg_alpha': 156.0}

In [32]:
# fist optimized <= baseline
xgb_regressor = xgb.XGBRegressor(
    n_estimators =115, 
    max_depth = 15, 
    gamma = 8.973856858008121,
    reg_alpha = 115,
    min_child_weight=41,
    colsample_bytree=0.4429943486665428)

# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(xgb_regressor, X, y, scoring='r2', cv=cv, n_jobs=-1)
scores

array([0.86813994, 0.84604672, 0.86711523, 0.83659513, 0.8405936 ,
       0.86901184, 0.67438086, 0.85746573, 0.86239255, 0.81523786,
       0.79464482, 0.75065495, 0.89896762, 0.85824837, 0.8382915 ,
       0.84777554, 0.83971453, 0.87611698, 0.84948332, 0.83367677,
       0.81939094, 0.85618041, 0.68354977, 0.88000769, 0.85802103,
       0.86859583, 0.86451091, 0.86623658, 0.89606794, 0.85252133])

In [33]:
np.mean(scores)

0.8389878760580836

In [ ]:
space = xgb_opt[0]
regressor = xgb.XGBRegressor(
    n_estimators =180, 
    max_depth = int(space['max_depth']), 
#     gamma = space['gamma'],
#     reg_alpha = int(space['reg_alpha']),
    min_child_weight=int(space['min_child_weight']),
    colsample_bytree=int(space['colsample_bytree']))
X_train, X_test, y_train, y_test = train_test_split(X, y)
regressor.fit(X_train, y_train)
# regressor

In [ ]:
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
# evaluate model
scores = cross_val_score(regressor, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
scores

In [ ]:
space = random_forest_opt[0]
regressor = RandomForestRegressor(
    n_estimators=int(space['n_estimators']),
    max_depth=int(space['max_depth']),
    min_samples_leaf=int(space['min_samples_leaf']),
    min_samples_split=int(space['min_samples_split']),
    n_jobs=-1
)
X_train, X_test, y_train, y_test = train_test_split(X_nonNa, y_nonNa)
regressor.fit(X_train, y_train)